In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

In [4]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device,
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    val_labels = sampler.test_data[sampler.test_mask] 

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None 

    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/756 [00:00<?, ?it/s]

epoch:   0 loss:0.710613 auc:0.6057
epoch:  20 loss:0.390960 auc:0.9487
epoch:  40 loss:0.375464 auc:0.9485
epoch:  60 loss:0.372162 auc:0.9498
epoch:  80 loss:0.370933 auc:0.9492
epoch: 100 loss:0.369972 auc:0.9492
epoch: 120 loss:0.368649 auc:0.9494
epoch: 140 loss:0.366055 auc:0.9490
epoch: 160 loss:0.360743 auc:0.9479


  0%|          | 1/756 [00:10<2:06:19, 10.04s/it]

epoch: 180 loss:0.354110 auc:0.9477
Fit finished.
epoch:   0 loss:0.784448 auc:0.4388
epoch:  20 loss:0.395113 auc:0.9320
epoch:  40 loss:0.377392 auc:0.9343
epoch:  60 loss:0.372745 auc:0.9341
epoch:  80 loss:0.371237 auc:0.9339
epoch: 100 loss:0.370002 auc:0.9341
epoch: 120 loss:0.368214 auc:0.9342
epoch: 140 loss:0.364917 auc:0.9344
epoch: 160 loss:0.360063 auc:0.9339
epoch: 180 loss:0.355371 auc:0.9344
epoch: 200 loss:0.351749 auc:0.9344


  0%|          | 2/756 [00:23<2:30:50, 12.00s/it]

...

epoch: 200 loss:0.347592 auc:0.9598
Fit finished.
epoch:   0 loss:0.814100 auc:0.4132
epoch:  20 loss:0.401617 auc:0.8589
epoch:  40 loss:0.377700 auc:0.8655
epoch:  60 loss:0.373417 auc:0.8689
epoch:  80 loss:0.371879 auc:0.8691
epoch: 100 loss:0.371115 auc:0.8690
epoch: 120 loss:0.370432 auc:0.8691
epoch: 140 loss:0.369573 auc:0.8688
epoch: 160 loss:0.368219 auc:0.8679
epoch: 180 loss:0.365830 auc:0.8664


100%|█████████▉| 755/756 [6:42:54<00:28, 28.53s/it]

epoch: 200 loss:0.362100 auc:0.8625
Fit finished.
epoch:   0 loss:0.686440 auc:0.6165
epoch:  20 loss:0.389037 auc:0.8454
epoch:  40 loss:0.374115 auc:0.8504
epoch:  60 loss:0.371334 auc:0.8511
epoch:  80 loss:0.369908 auc:0.8512
epoch: 100 loss:0.368319 auc:0.8511
epoch: 120 loss:0.365261 auc:0.8504
epoch: 140 loss:0.359531 auc:0.8467
epoch: 160 loss:0.352526 auc:0.8471
epoch: 180 loss:0.344504 auc:0.8465
epoch: 200 loss:0.333013 auc:0.8521
epoch: 220 loss:0.322290 auc:0.8544
epoch: 240 loss:0.311787 auc:0.8547
epoch: 260 loss:0.304316 auc:0.8680


100%|██████████| 756/756 [6:43:06<00:00, 31.99s/it]

epoch: 280 loss:0.294550 auc:0.8491
Fit finished.


In [7]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")